In [1]:
from dj.construction.extract import extract_dependencies
from dj.construction.inference import get_type_of_expression
from dj.sql.parsing import ast

In [2]:
from dj.utils import get_session
session = next(get_session())

In [3]:
tree, deps, danglers = extract_dependencies(session, """
    select Sum(revenue) from purchases_over_a_grand where revenue>1000.0 group by country
    """, raise_=False)

print("All Column types ", [(c, c.type) for c in list(tree.find_all(ast.Column))])
print()
print("Column Expressions", [(c, c.expression) for c in list(tree.find_all(ast.Column))])
print()
print("Projection Exp types ", [(exp, get_type_of_expression(exp)) for exp in tree.select.projection])
print()
deps, danglers

All Column types  [(Column(name=Name(name='country', quote_style=''), namespace=None), <ColumnType.STR: 'STR'>), (Column(name=Name(name='revenue', quote_style=''), namespace=None), <ColumnType.FLOAT: 'FLOAT'>), (Column(name=Name(name='revenue', quote_style=''), namespace=None), <ColumnType.FLOAT: 'FLOAT'>)]

Column Expressions [(Column(name=Name(name='country', quote_style=''), namespace=None), None), (Column(name=Name(name='revenue', quote_style=''), namespace=None), None), (Column(name=Name(name='revenue', quote_style=''), namespace=None), None)]

Projection Exp types  [(Function(name=Name(name='Sum', quote_style=''), namespace=Namespace(names=[]), args=[Column(name=Name(name='revenue', quote_style=''), namespace=None)]), <ColumnType.FLOAT: 'FLOAT'>)]



({Node(name='purchases_over_a_grand', type=<NodeType.TRANSFORM: 'transform'>, id=23, query='select country, revenue from revenue where revenue>1000.0', created_at=datetime.datetime(2023, 1, 12, 14, 19, 12, 301204), updated_at=datetime.datetime(2023, 1, 12, 14, 19, 12, 301210), description='', columns=[Column(id=129, name='country', dimension_column=None, dimension_id=None, type=<ColumnType.STR: 'STR'>), Column(id=130, name='revenue', dimension_column=None, dimension_id=None, type=<ColumnType.FLOAT: 'FLOAT'>)])},
 set())

In [4]:
def name_exp(exp: ast.Expression, default: str):
    if isinstance(exp, ast.Alias):
        return make_name(exp.namespace, exp.name.name)
    return default

In [5]:
{name_exp(exp, f"_col{i}"): get_type_of_expression(exp) for i, exp in enumerate(tree.select.projection)}

{'_col0': <ColumnType.FLOAT: 'FLOAT'>}

In [6]:
tree, deps, danglers = extract_dependencies(session,     """
    select Count(*) from purchases_over_a_grand where revenue>1000.0 group by country
    """, raise_=False)

print("All Column types ", [(c, c.type) for c in list(tree.find_all(ast.Column))])
print()
print("Column Expressions", [(c, c.expression) for c in list(tree.find_all(ast.Column))])
print()
print("Projection Exp types ", [(exp, get_type_of_expression(exp)) for exp in tree.select.projection])
print()
deps, danglers

All Column types  [(Column(name=Name(name='country', quote_style=''), namespace=None), <ColumnType.STR: 'STR'>), (Column(name=Name(name='revenue', quote_style=''), namespace=None), <ColumnType.FLOAT: 'FLOAT'>)]

Column Expressions [(Column(name=Name(name='country', quote_style=''), namespace=None), None), (Column(name=Name(name='revenue', quote_style=''), namespace=None), None)]

Projection Exp types  [(Function(name=Name(name='Count', quote_style=''), namespace=Namespace(names=[]), args=[Wildcard()]), <ColumnType.INT: 'INT'>)]



({Node(name='purchases_over_a_grand', type=<NodeType.TRANSFORM: 'transform'>, id=23, query='select country, revenue from revenue where revenue>1000.0', created_at=datetime.datetime(2023, 1, 12, 14, 19, 12, 301204), updated_at=datetime.datetime(2023, 1, 12, 14, 19, 12, 301210), description='', columns=[Column(id=129, name='country', dimension_column=None, dimension_id=None, type=<ColumnType.STR: 'STR'>), Column(id=130, name='revenue', dimension_column=None, dimension_id=None, type=<ColumnType.FLOAT: 'FLOAT'>)])},
 set())

In [7]:
tree, deps, danglers = extract_dependencies(session, """
    select sum_revenue from (select Sum(revenue) as sum_revenue from purchases_over_a_grand where revenue>1000.0 group by country)
    """, raise_=False)

print("All Column types ", [(c, c.type) for c in list(tree.find_all(ast.Column))])
print()
print("Column Expressions", [(c, c.expression) for c in list(tree.find_all(ast.Column))])
print()
print("Projection Exp types ", [(exp, get_type_of_expression(exp)) for exp in tree.select.projection])
print()
deps, danglers

All Column types  [(Column(name=Name(name='country', quote_style=''), namespace=None), <ColumnType.STR: 'STR'>), (Column(name=Name(name='revenue', quote_style=''), namespace=None), <ColumnType.FLOAT: 'FLOAT'>), (Column(name=Name(name='revenue', quote_style=''), namespace=None), <ColumnType.FLOAT: 'FLOAT'>), (Column(name=Name(name='sum_revenue', quote_style=''), namespace=None), None)]

Column Expressions [(Column(name=Name(name='country', quote_style=''), namespace=None), None), (Column(name=Name(name='revenue', quote_style=''), namespace=None), None), (Column(name=Name(name='revenue', quote_style=''), namespace=None), None), (Column(name=Name(name='sum_revenue', quote_style=''), namespace=None), Alias(name=Name(name='sum_revenue', quote_style=''), namespace=None, child=Function(name=Name(name='Sum', quote_style=''), namespace=Namespace(names=[]), args=[Column(name=Name(name='revenue', quote_style=''), namespace=None)])))]

Projection Exp types  [(Column(name=Name(name='sum_revenue', q

({Node(name='purchases_over_a_grand', type=<NodeType.TRANSFORM: 'transform'>, id=23, query='select country, revenue from revenue where revenue>1000.0', created_at=datetime.datetime(2023, 1, 12, 14, 19, 12, 301204), updated_at=datetime.datetime(2023, 1, 12, 14, 19, 12, 301210), description='', columns=[Column(id=129, name='country', dimension_column=None, dimension_id=None, type=<ColumnType.STR: 'STR'>), Column(id=130, name='revenue', dimension_column=None, dimension_id=None, type=<ColumnType.FLOAT: 'FLOAT'>)])},
 set())

In [8]:
tree, deps, danglers = extract_dependencies(session, """
    select sum_revenue from (select Sum(revenue) as sum_revenue from purchases_over_a_grand where revenue>1000.0 group by country) as a, oops
    """, raise_=False)

print("All Column types ", [(c, c.type) for c in list(tree.find_all(ast.Column))])
print()
print("Column Expressions", [(c, c.expression) for c in list(tree.find_all(ast.Column))])
print()
print("Projection Exp types ", [(exp, get_type_of_expression(exp)) for exp in tree.select.projection])
print()
deps, danglers

All Column types  [(Column(name=Name(name='country', quote_style=''), namespace=None), <ColumnType.STR: 'STR'>), (Column(name=Name(name='revenue', quote_style=''), namespace=None), <ColumnType.FLOAT: 'FLOAT'>), (Column(name=Name(name='revenue', quote_style=''), namespace=None), <ColumnType.FLOAT: 'FLOAT'>), (Column(name=Name(name='sum_revenue', quote_style=''), namespace=None), None)]

Column Expressions [(Column(name=Name(name='country', quote_style=''), namespace=None), None), (Column(name=Name(name='revenue', quote_style=''), namespace=None), None), (Column(name=Name(name='revenue', quote_style=''), namespace=None), None), (Column(name=Name(name='sum_revenue', quote_style=''), namespace=None), Alias(name=Name(name='sum_revenue', quote_style=''), namespace=None, child=Function(name=Name(name='Sum', quote_style=''), namespace=Namespace(names=[]), args=[Column(name=Name(name='revenue', quote_style=''), namespace=None)])))]

Projection Exp types  [(Column(name=Name(name='sum_revenue', q

({Node(name='purchases_over_a_grand', type=<NodeType.TRANSFORM: 'transform'>, id=23, query='select country, revenue from revenue where revenue>1000.0', created_at=datetime.datetime(2023, 1, 12, 14, 19, 12, 301204), updated_at=datetime.datetime(2023, 1, 12, 14, 19, 12, 301210), description='', columns=[Column(id=129, name='country', dimension_column=None, dimension_id=None, type=<ColumnType.STR: 'STR'>), Column(id=130, name='revenue', dimension_column=None, dimension_id=None, type=<ColumnType.FLOAT: 'FLOAT'>)])},
 {'oops'})